In [3]:
!pip install --upgrade langchain

In [4]:
!pip install --upgrade openai

In [22]:
!pip install --upgrade tiktoken

In [ ]:
api_key = <key>
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = <pw>
import os
os.environ["OPENAI_API_KEY"] = api_key

In [16]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

In [17]:
llm = OpenAI()

template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
""")

llm_chain = LLMChain(
    llm=llm,
    prompt=template,
    output_parser=StrOutputParser()
)

response = llm_chain.run(fruit="apple")

print(response)


Well, apples is pears, mate! They're nice and juicy, and perfect for a snack. I've got a few Grannies and a few Coxes if you're interested.


In [10]:
response = llm("What is Neo4j?")

In [98]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, RetrievalQA
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool, YouTubeSearchTool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.schema import StrOutputParser
from langchain.output_parsers.json import SimpleJsonOutputParser

llm = ChatOpenAI()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

prompt = PromptTemplate(
    template="""
    You are a legal expert for bankers. You need to answer questions about FFIEC rules.
    Short answers are great.
    Answer should be valid text only. No json.
    ChatHistory:{chat_history}
    Question:{input}
    """,
    input_variables=["chat_history", "input"]
    )

chat_chain = LLMChain(llm=llm, prompt=prompt, memory=memory, output_parser=StrOutputParser(), verbose=True)
#vector_chain = LLMChain(llm=llm, prompt=prompt, memory=memory, output_parser=SimpleJsonOutputParser(), verbose=True)

embedding_provider = OpenAIEmbeddings()

ffiec_paragraph_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
    index_name="ffiec_text",
    embedding_node_property="text_v_embedding",
    text_node_property="paragraph_text"
)

retrievalQA = RetrievalQA.from_llm(
    llm=llm,
    retriever=ffiec_paragraph_vector.as_retriever(),
    verbose=True,
    return_source_documents=True
)

def run_retriever(query):
    results = retrievalQA({"query":query})
    return str(results)

tools = [
    Tool.from_function(
        name="ChatOpenAI",
        description="For when you need to chat about rules or laws. The question will be a string. Return a string.",
        func=chat_chain.run,
        return_direct=True
    ),
    Tool.from_function(
        name="ParagraphRetrieval",
        description="For when you need to compare a FFIEC rule. The question will be a string. Return a string.",
        func=run_retriever,
        return_direct=True
    )
]

agent = initialize_agent(
    tools, llm, memory=memory,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

while True:
    q = input(">")
    print(agent.invoke({"input": q}))
    break

/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/langchain/vectorstores/neo4j_vector.py:167: ExperimentalWarning: The configuration may change in the future.
  self._driver.verify_connectivity()


>Strategic plans that include significant market expansion or the addition of new products and services may expose financial institutions to increased risks. Which ones ?


> Entering new AgentExecutor chain...
{
    "action": "ParagraphRetrieval",
    "action_input": "Strategic plans that include significant market expansion or the addition of new products and services may expose financial institutions to increased risks."
}

> Entering new RetrievalQA chain...

> Finished chain.

Observation: {'query': 'Strategic plans that include significant market expansion or the addition of new products and services may expose financial institutions to increased risks.', 'result': "Yes, strategic plans that involve significant market expansion or the addition of new products and services can expose financial institutions to increased risks. This is because entering new markets or introducing new products and services often involves uncertainties and challenges that can impact the institution's e

In [101]:
print("Yes, strategic plans that involve significant market expansion or the addition of new products and services can expose financial institutions to increased risks. This is because entering new markets or introducing new products and services often involves uncertainties and challenges that can impact the institution's earnings, capital, and reputation. It is important for financial institutions to carefully assess and manage these risks as part of their strategic planning process.")

Yes, strategic plans that involve significant market expansion or the addition of new products and services can expose financial institutions to increased risks. This is because entering new markets or introducing new products and services often involves uncertainties and challenges that can impact the institution's earnings, capital, and reputation. It is important for financial institutions to carefully assess and manage these risks as part of their strategic planning process.


In [110]:
vector_index = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
    index_name="ffiec_text",
    embedding_node_property="text_v_embedding",
    text_node_property="paragraph_text"
)


/Users/pierrehalftermeyer/opt/anaconda3/lib/python3.8/site-packages/langchain/vectorstores/neo4j_vector.py:167: ExperimentalWarning: The configuration may change in the future.
  self._driver.verify_connectivity()


In [116]:
question ="""Strategic plans that include significant market expansion or the addition of new products and
services may expose financial institutions to increased risks. Which ones ?"""

In [117]:
response = vector_index.similarity_search(question)
print(response[0].page_content)

Strategic risk is associated with the financial institution's mission and future business plans.  This risk category includes plans for entering new business lines, expanding existing services through mergers and acquisitions, and enhancing infrastructure (e.g., physical plant and equipment, IT, and networking).  The variety of emerging technologies for retail payments demands integration of payment strategies into the financial institution's overall strategic planning processes.  Financial institutions also compete increasingly with highly innovative nonbank entities to provide retail payment services.  This competition benefits the consumer through enhanced product offerings at a lower cost.  Conversely, competition places additional pressure on financial institutions to protect profitability through the development of new products and services while managing additional marketing, research, and development costs.
Strategic plans that include significant market expansion or the additi

In [118]:
vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(), chain_type="stuff", retriever=vector_index.as_retriever(k=1))

print(
    vector_qa.run(question)
)


The increased risks that financial institutions may be exposed to when implementing strategic plans that include significant market expansion or the addition of new products and services include:

1. Strategic risk: This refers to the risk associated with the financial institution's mission and future business plans. It includes risks related to entering new business lines, expanding existing services through mergers and acquisitions, and enhancing infrastructure. For example, expanding Internet banking services to include electronic bill presentment and payment services or entering the merchant bankcard processing business significantly increases the potential risk to the financial institution.

2. Compliance (legal) risk: When financial institutions expand their services or enter new markets, they may face increased compliance risk. This risk arises when outsourced activities fail to comply with legal or regulatory requirements, exposing the institution to legal sanctions.

3. Reputa